In [1]:
import unicodecsv
import django
import codecs
import json
import re
django.setup()
from sefaria.model import *
from collections import defaultdict
from itertools import chain

WARNING Failed to load 're2'.  Falling back to 're' for regular expression parsing. See https://github.com/Sefaria/Sefaria-Project/wiki/Regular-Expression-Engines
WARNING Failed to load 're2'.  Falling back to 're' for regular expression parsing. See https://github.com/blockspeiser/Sefaria-Project/wiki/Regular-Expression-Engines


In [14]:
sef_mis = []
sef_tal = []
gens = TimePeriodSet.get_generations()
for gen in gens:
    people = gen.get_people_in_generation()
    for p in people:
        
        d = {
            'aliases': {'he': [], 'en': []},
            '_id': str(p._id),
            'edges': {v['en'].lower(): [{'id': str(q._id), 'name': next(name['text'] for name in q.names if name.get('primary', False) and name['lang'] == 'en'), 'heName': next(name['text'] for name in q.names if name.get('primary', False) and name['lang'] == 'he')} for q in v['people']] for k, v in p.get_grouped_relationships().items()},
            'jeLink': p.jeLink,
            'enWikiLink': getattr(p, 'enWikiLink', None),
            'heWikiLink': getattr(p, 'heWikiLink', None),
            'generation': gen.symbol
        }
        print d['edges']
        for n in p.names:
            if n.get('primary', False):
                if n['lang'] == 'en':
                    d['name'] = n['text']
                elif n['lang'] == 'he':
                    d['heName'] = n['text']
            else:
                d['aliases'][n['lang']] += [u' '.join(n['text'])]
        if gen.symbol.startswith('A'):
            sef_tal += [d]
        else:
            sef_mis += [d]
print len(sef_mis)
print len(sef_tal)
print yo

{u'learned from': [{'heName': u'\u05d0\u05e0\u05d8\u05d9\u05d2\u05e0\u05d5\u05e1 \u05d0\u05d9\u05e9 \u05e1\u05d5\u05db\u05d5', 'id': '565584126532d12b35ca6dda', 'name': u'Antigonus of Sokho'}, {'heName': u'\u05e9\u05de\u05e2\u05d5\u05df \u05d4\u05e6\u05d3\u05d9\u05e7', 'id': '565584126532d12b35ca6e8e', 'name': u'Shimon haTzaddik'}], u'taught': [{'heName': u'\u05d9\u05d4\u05d5\u05e9\u05e2 \u05d1\u05df \u05e4\u05e8\u05d7\u05d9\u05d4\u200e', 'id': '565584126532d12b35ca6e23', 'name': u'Yehoshua b. Perachya'}, {'heName': u'\u05e0\u05ea\u05d0\u05d9 \u05d4\u05d0\u05e8\u05d1\u05dc\u05d9', 'id': '565584126532d12b35ca6e56', 'name': u'Nittai haArbeli'}]}
{u'learned from': [{'heName': u'\u05d0\u05e0\u05d8\u05d9\u05d2\u05e0\u05d5\u05e1 \u05d0\u05d9\u05e9 \u05e1\u05d5\u05db\u05d5', 'id': '565584126532d12b35ca6dda', 'name': u'Antigonus of Sokho'}, {'heName': u'\u05e9\u05de\u05e2\u05d5\u05df \u05d4\u05e6\u05d3\u05d9\u05e7', 'id': '565584126532d12b35ca6e8e', 'name': u'Shimon haTzaddik'}], u'taught': [{

{}
{}
{}
{}
{u'learned from': [{'heName': u'\u05e8\u05d1\u05df \u05d9\u05d5\u05d7\u05e0\u05df \u05d1\u05df \u05d6\u05db\u05d0\u05d9\u200e', 'id': '565584126532d12b35ca6e27', 'name': u'Rabbi Yohanan b. Zakkai'}]}
{}
{u'learned from': [{'heName': u'\u05e8\u05d1\u05df \u05d9\u05d5\u05d7\u05e0\u05df \u05d1\u05df \u05d6\u05db\u05d0\u05d9\u200e', 'id': '565584126532d12b35ca6e27', 'name': u'Rabbi Yohanan b. Zakkai'}]}
{u'learned from': [{'heName': u'\u05e8\u05d1\u05d9 \u05d0\u05dc\u05d9\u05e2\u05d6\u05e8 \u05d1\u05df \u05d4\u05d5\u05e8\u05e7\u05e0\u05d5\u05e1\u200e', 'id': '565584126532d12b35ca6dca', 'name': u'Rabbi Eliezer b. Hyrcanus'}, {'heName': u'\u05e8\u05d1\u05d9 \u05d9\u05d4\u05d5\u05e9\u05e2 \u05d1\u05df \u05d7\u05e0\u05e0\u05d9\u05d4', 'id': '565584126532d12b35ca6e21', 'name': u'Rabbi Yehoshua b. Hananyah'}, {'heName': u'\u05e8\u05d1\u05d9 \u05e2\u05e7\u05d9\u05d1\u05d0', 'id': '565584126532d12b35ca6e5e', 'name': u'Rabbi Akiva'}]}
{u'taught': [{'heName': u'\u05e8\u05d1\u05d9 \u05de\

{u'learned from': [{'heName': u'\u05e8\u05d1\u05d9 \u05d0\u05dc\u05e2\u05d6\u05e8 \u05d1\u05df \u05e9\u05de\u05d5\u05e2', 'id': '565584126532d12b35ca6dd4', 'name': u"Rabbi Elazar b. Shamu'a"}, {'heName': u'\u05e8\u05d1\u05d9 \u05d9\u05d4\u05d5\u05d3\u05d4 \u05d1\u05e8\u05d1\u05d9 \u05d0\u05d9\u05dc\u05e2\u05d0\u05d9', 'id': '565584126532d12b35ca6e1d', 'name': u"Rabbi Yehudah b. R' II'ai"}, {'heName': u'\u05e8\u05d1\u05d9 \u05d9\u05d5\u05e1\u05d9 \u05d1\u05df \u05d7\u05dc\u05e4\u05ea\u05d0', 'id': '565584126532d12b35ca6e2e', 'name': u'Rabbi Yose b. Halafta'}, {'heName': u'\u05e8\u05d1\u05d9 \u05de\u05d0\u05d9\u05e8', 'id': '565584126532d12b35ca6e48', 'name': u'Rabbi Meir'}, {'heName': u'\u05e8\u05d1\u05d9 \u05e0\u05ea\u05df \u05d4\u05d1\u05d1\u05dc\u05d9', 'id': '565584126532d12b35ca6e57', 'name': u'Rabbi Natan haBavli'}, {'heName': u'\u05e8\u05d1\u05df \u05e9\u05de\u05e2\u05d5\u05df \u05d1\u05df \u05d2\u05de\u05dc\u05d9\u05d0\u05dc', 'id': '565584126532d12b35ca6e81', 'name': u'Rabban S

{u'taught': [{'heName': u'\u05e8\u05d1 \u05d7\u05e1\u05d3\u05d0', 'id': '565584126532d12b35ca6e14', 'name': u'Rav Hisda'}]}
{u'learned from': [{'heName': u'\u05e8\u05d1', 'id': '565584126532d12b35ca6e67', 'name': u'Rav'}], u'taught': [{'heName': u'\u05e8\u05d1 \u05d4\u05de\u05e0\u05d5\u05e0\u05d0', 'id': '565584126532d12b35ca6df6', 'name': u'Rav Hamnuna (II)'}, {'heName': u'\u05e8\u05d1 \u05e0\u05d7\u05de\u05df \u05d1\u05e8 \u05d9\u05e2\u05e7\u05d1', 'id': '565584126532d12b35ca6e54', 'name': u"Rav Nahman b. Ya'akov"}]}
{u'learned from': [{'heName': u'\u05e8\u05d1', 'id': '565584126532d12b35ca6e67', 'name': u'Rav'}]}
{u'learned from': [{'heName': u'\u05e8\u05d1', 'id': '565584126532d12b35ca6e67', 'name': u'Rav'}]}
{u'learned from': [{'heName': u'\u05e8\u05d1', 'id': '565584126532d12b35ca6e67', 'name': u'Rav'}, {'heName': u'\u05e9\u05de\u05d5\u05d0\u05dc', 'id': '565584126532d12b35ca6e79', 'name': u'Shmuel'}], u'taught': [{'heName': u'\u05e8\u05d1 \u05d0\u05d7\u05d0 \u05d1\u05e8 \u05d9\u

{u'learned from': [{'heName': u'\u05e8\u05d1\u05d9 \u05d9\u05d5\u05d7\u05e0\u05df \u05d1\u05e8 \u05e0\u05e4\u05d7\u05d0', 'id': '565584126532d12b35ca6e29', 'name': u'Rabbi Yohanan b. Nappaha'}, {'heName': u'\u05e8\u05d1\u05d9 \u05e9\u05de\u05e2\u05d5\u05df \u05d1\u05df \u05dc\u05e7\u05d9\u05e9', 'id': '565584126532d12b35ca6e86', 'name': u'Rabbi Shimon b. Lakish'}], u'taught': [{'heName': u'\u05e8\u05d1 \u05d4\u05de\u05e0\u05d5\u05e0\u05d0', 'id': '565584126532d12b35ca6df6', 'name': u'Rav Hamnuna (II)'}]}
{u'learned from': [{'heName': u'\u05e8\u05d1', 'id': '565584126532d12b35ca6e67', 'name': u'Rav'}]}
{u'learned from': [{'heName': u'\u05e8\u05d1', 'id': '565584126532d12b35ca6e67', 'name': u'Rav'}], u'taught': [{'heName': u'\u05e8\u05d1 \u05e0\u05d7\u05de\u05df \u05d1\u05e8 \u05d9\u05e2\u05e7\u05d1', 'id': '565584126532d12b35ca6e54', 'name': u"Rav Nahman b. Ya'akov"}]}
{u'learned from': [{'heName': u'\u05e8\u05d1\u05d9 \u05d9\u05d4\u05d5\u05d3\u05d4 \u05e0\u05e9\u05d9\u05d0\u05d4', 'id'

{u'learned from': [{'heName': u'\u05e8\u05d1\u05d9 \u05d0\u05d1\u05d4\u05d5\u200e', 'id': '565584126532d12b35ca6dba', 'name': u'Rabbi Abbahu'}, {'heName': u'\u05e8\u05d1 \u05d0\u05d3\u05d0 \u05d1\u05e8 \u05d0\u05d4\u05d1\u05d4', 'id': '565584126532d12b35ca6dc2', 'name': u'Rav Adda b. Ahavah (I)'}, {'heName': u'\u05e8\u05d1\u05d4 \u05d1\u05e8 \u05d0\u05d1\u05d5\u05d4', 'id': '565584126532d12b35ca6e69', 'name': u'Rabbah b. Avuha'}], u'taught': [{'heName': u'\u05d0\u05d1\u05d9\u05d9\u200e', 'id': '565584126532d12b35ca6dbd', 'name': u'Abaye'}, {'heName': u'\u05e8\u05d1 \u05d0\u05d7\u05d0 \u05d1\u05e8 \u05d9\u05e2\u05e7\u05d1', 'id': '565584126532d12b35ca6dc6', 'name': u"Rav Aha b. Ya'akov"}, {'heName': u'\u05e8\u05d1 \u05d3\u05d9\u05de\u05d9', 'id': '565584126532d12b35ca6deb', 'name': u'Rav Dimi'}, {'heName': u'\u05e8\u05d1 \u05e0\u05d7\u05de\u05df \u05d1\u05e8 \u05d9\u05e6\u05d7\u05e7', 'id': '565584126532d12b35ca6e55', 'name': u'Rav Nahman b. Yitzhak'}, {'heName': u'\u05e8\u05d1\u05d0', 

{u'learned from': [{'heName': u'\u05e8\u05d1 \u05d7\u05e1\u05d3\u05d0', 'id': '565584126532d12b35ca6e14', 'name': u'Rav Hisda'}, {'heName': u'\u05e8\u05d1 \u05d9\u05d5\u05e1\u05e3 \u05d1\u05e8 \u05d4\u05d9\u05d9\u05d0', 'id': '565584126532d12b35ca6e34', 'name': u'Rav Yosef b. Hiyya'}, {'heName': u'\u05e8\u05d1 \u05e0\u05d7\u05de\u05df \u05d1\u05e8 \u05d9\u05e2\u05e7\u05d1', 'id': '565584126532d12b35ca6e54', 'name': u"Rav Nahman b. Ya'akov"}], u'taught': [{'heName': u'\u05e8\u05d1 \u05d0\u05d3\u05d0 \u05d1\u05e8 \u05d0\u05d4\u05d1\u05d4', 'id': '565584126532d12b35ca6dc1', 'name': u'Rav Adda b. Ahavah (II)'}, {'heName': u'\u05e8\u05d1 \u05d2\u05d1\u05d9\u05d4\u05d4 \u05de\u05d1\u05d9 \u05db\u05ea\u05d9\u05dc', 'id': '565584126532d12b35ca6de4', 'name': u'Rav Geviha of Bei Katil'}, {'heName': u'\u05e8\u05d1 \u05d6\u05d1\u05d9\u05d3', 'id': '565584126532d12b35ca6df9', 'name': u'Rav Zevid'}, {'heName': u'\u05e8\u05d1 \u05db\u05d4\u05e0\u05d0', 'id': '565584126532d12b35ca6e45', 'name': u'Rav 

In [3]:
with open("aspaklaria_people.csv", "rb") as fin:
    csv = unicodecsv.DictReader(fin)
    asp = []
    for row in csv:
        asp += [row]
    asp_bib = filter(lambda x: x['category1'] == 'biblical person' or x['category2'] == 'biblical person', asp)
    asp_tal = filter(lambda x: x['category1'] == 'talmudic person' or x['category2'] == 'talmudic person', asp)
    asp_mis = filter(lambda x: x['category1'] == 'mishnaic person' or x['category2'] == 'mishnaic person', asp)
for yo in asp_mis:
    print yo['he']

אונקלוס
אותו האיש
אחר אלישע בן אבויה
אחר
אימא שלום
אלישע בן אבויה
אלכסנדר
אנדרינוס
אנשי כנסת הגדולה
אספסינוס
בן סטדא
בן סירא
בן ציצית הכסת
בן תמליון
בנימין הצדיק
בר כוכבא
ברוריה
גביעא בן פסיסא
הורדוס
הורקנוס רבי אליעזר
הורקנוס
הלל
חוני המעגל
חנה ושבעת בניה
טבי
טורנוס רופוס
טיטוס
יהודה בן גרים
יהודה המכבי
יהודית
יוחנן כהן גדול
יוסף בן סימאי
יוסף הכהן
ינאי
יקום איש צרורות
ישמעאל בן פיאבי
יששכר איש כפר ברקאי
כלבא שבוע
לולינוס ופפוס
מונבז
מרים בת בייתוס
מרים בת בלגה
מרים בת נקדימון
מרים התרמודית
מרתא בת בייתוס
מתתיהו
נחום איש גמזו
ניקנור
נקדימון
עקילס
ציצית הכסת
קטיעה בר שלום
קליאופטרא
קמחית
אבא בנימין
אבא חלפתא
אבא חלקיה
אבא יודן
אבא שאול אבא שאול בן בטנית
אבא תחנה
אדמון
איסי בר יהודה
בבא בן בוטא
בן זומא
בן ננס
בן עזאי
בני בתירא
בנימין הצדיק
בר קפרא
ברוריה
הורקנוס
הלל בן רבן גמליאל
הלל
חוני המעגל
חוצפית המתורגמן
חנן בן אבישלום
חנן המצרי
חנן הנחבא
יהודה בן דורתאי
יהושע בן גמלא
יוחנן בן בג בג
יוחנן כהן גדול
יונתן בן עוזיאל
יונתן בן עמרם
יוסי בן יועזר
יוסי בן סימאי
לוי בן סיסי
נחום איש גם זו

In [15]:
def normalize_rabbi(rabbi):
    rabbi = re.sub(ur'^(?:ר|רב|רבי|רבן) ', u'', rabbi)
    rabbi = re.sub(ur' בר ',u' בן ', rabbi)
    rabbi = re.sub(ur'([\u05d0-\u05ea])\u05d0(?= |$)', ur'\1\u05d4', rabbi)
    rabbi = re.sub(ur'[^a-zA-Z\u05d0-\u05ea\'"״׳., ]', u'', rabbi)
    return rabbi

sef_mis_full_name = defaultdict(list)
sef_mis_first_name = defaultdict(list)
sef_mis_alias = defaultdict(list)
sef_mis_all = []
for sef_guy in sef_mis:
    item = {'id': str(sef_guy['_id']), 'name': sef_guy['name'], 'heName': sef_guy['heName'], 'matched': False, 'jeLink': sef_guy['jeLink'], 'heWikiLink': sef_guy['heWikiLink'], 'enWikiLink': sef_guy['enWikiLink'], 'generation': sef_guy['generation']}
    sef_mis_all += [item]
    heName = normalize_rabbi(sef_guy['heName'])
    if not any([x['id'] == item['id'] for x in sef_mis_full_name[heName]]):
        sef_mis_full_name[heName] += [item]
    first_name = heName.split()[0]

    if not any([x['id'] == item['id'] for x in sef_mis_first_name[first_name]]):
        sef_mis_first_name[first_name] += [item]
    for alias in sef_guy['aliases'].get('he', []):
        if not any([x['id'] == item['id'] for x in sef_mis_alias[alias]]):
            sef_mis_alias[alias] += [item]
asp_mis_map = {}
for asp_guy in asp_mis:
    full_name = normalize_rabbi(asp_guy['he'])
    first_name = full_name.split()[0]
    full_name_match = sef_mis_full_name.get(full_name, [])
    first_name_match = sef_mis_first_name.get(first_name, [])
    alias_match = sef_mis_alias.get(full_name, [])
    if first_name != full_name:
        alias_match += sef_mis_alias.get(first_name, [])
    for yo in full_name_match:
        yo['matched'] = True
    for yo in first_name_match:
        yo['matched'] = True
    for yo in alias_match:
        yo['matched'] = True
    asp_mis_map[asp_guy['he']] = {'full_name': full_name_match, 'first_name': first_name_match, 'alias': alias_match}

In [16]:
sef_tal_full_name = defaultdict(list)
sef_tal_first_name = defaultdict(list)
sef_tal_alias = defaultdict(list)
sef_tal_all = [] 
for sef_guy in sef_tal:
    print sef_guy
    item = {'id': str(sef_guy['_id']), 'name': sef_guy['name'], 'heName': sef_guy['heName'], 'matched': False, 'jeLink': sef_guy['jeLink'], 'heWikiLink': sef_guy['heWikiLink'], 'enWikiLink': sef_guy['enWikiLink'], 'generation': sef_guy['generation']}
    sef_tal_all += [item]
    heName = normalize_rabbi(sef_guy['heName'])
    
    if not any([x['id'] == item['id'] for x in sef_tal_full_name[heName]]):
        sef_tal_full_name[heName] += [item]
    first_name = heName.split()[0]

    if not any([x['id'] == item['id'] for x in sef_tal_first_name[first_name]]):
        sef_tal_first_name[first_name] += [item]
    for alias in sef_guy['aliases'].get('he', []):
        if not any([x['id'] == item['id'] for x in sef_tal_alias[alias]]):
            sef_tal_alias[alias] += [item]
asp_tal_map = {}
for asp_guy in asp_tal:
    full_name = normalize_rabbi(asp_guy['he'])
    first_name = full_name.split()[0]
    full_name_match = sef_tal_full_name.get(full_name, [])
    first_name_match = sef_tal_first_name.get(first_name, [])
    alias_match = sef_tal_alias.get(full_name, [])
    if first_name != full_name:
        alias_match += sef_tal_alias.get(first_name, [])
    for yo in full_name_match:
        yo['matched'] = True
    for yo in first_name_match:
        yo['matched'] = True
    for yo in alias_match:
        yo['matched'] = True
    asp_tal_map[asp_guy['he']] = {'full_name': full_name_match, 'first_name': first_name_match, 'alias': alias_match}

{'_id': '565584126532d12b35ca6dbb', 'name': u'Avuha Di Shmuel', 'generation': u'A1', 'heWikiLink': None, 'jeLink': None, 'edges': {u'taught': [{'heName': u'\u05e9\u05de\u05d5\u05d0\u05dc', 'id': '565584126532d12b35ca6e79', 'name': u'Shmuel'}]}, 'heName': u'\u05d0\u05d1\u05d5\u05d4 \u05d3\u05e9\u05de\u05d5\u05d0\u05dc', 'enWikiLink': None, 'aliases': {'en': [u' Abba b. Abba', u' Avuha diShmuel', u' Avuha the father of Shmuel'], 'he': [u' \u05d0\u05d1\u05d0 \u05d1\u05e8 \u05d0\u05d1\u05d0']}}
{'_id': '565584126532d12b35ca6ddb', 'name': u'Rav Assi', 'generation': u'A1', 'heWikiLink': None, 'jeLink': u'http://www.jewishencyclopedia.com/view.jsp?artid=2038&letter=A#6171', 'edges': {}, 'heName': u'\u05e8\u05d1 \u05d0\u05e1\u05d9', 'enWikiLink': None, 'aliases': {'en': [], 'he': []}}
{'_id': '565584126532d12b35ca6ddd', 'name': u'Rabbi Aphes', 'generation': u'A1', 'heWikiLink': None, 'jeLink': None, 'edges': {}, 'heName': u'\u05e8\u05d1\u05d9 \u05d0\u05e4\u05e1', 'enWikiLink': None, 'aliases':

In [6]:
sef_tal_first_name[u'שמואל']

[{'heName': u'\u05e9\u05de\u05d5\u05d0\u05dc',
  'id': '565584126532d12b35ca6e79',
  'matched': True,
  'name': u'Shmuel'},
 {'heName': u'\u05e8\u05d1\u05d9 \u05e9\u05de\u05d5\u05d0\u05dc \u05d1\u05df \u05d9\u05e6\u05d7\u05e7',
  'id': '565584126532d12b35ca6e7a',
  'matched': True,
  'name': u'Rabbi Shmuel b. Yitzhak'},
 {'heName': u'\u05e8\u05d1\u05d9 \u05e9\u05de\u05d5\u05d0\u05dc \u05d1\u05df \u05e0\u05d7\u05de\u05df',
  'id': '565584126532d12b35ca6e7b',
  'matched': True,
  'name': u'Rabbi Shmuel b. Nahmani'},
 {'heName': u'\u05e8\u05d1\u05d9 \u05e9\u05de\u05d5\u05d0\u05dc \u05d1\u05e8 \u05d9\u05d5\u05e1\u05d9 \u05d1\u05e8 \u05d1\u05d5\u05df',
  'id': '565584126532d12b35ca6e7c',
  'matched': True,
  'name': u'Rabbi Shmuel b. Yose b. Boon'}]

In [12]:
# Aspaklaria matched / unmatched

rows = []
max_match = -1
all_items = []
for name, val in chain(asp_mis_map.items(), asp_tal_map.items()):
    item = {'Name': name}
    matches = val['full_name']
    if len(matches) == 0:
        matches = val['first_name']
    if len(matches) == 0:
        matches = val['alias']
    ids_seen = set()
    for i, m in enumerate(matches):
        if m['id'] in ids_seen:
            continue
        ids_seen.add(m['id'])
        item['Match Name {}'.format(i+1)] = m['heName']
        item['Match URL {}'.format(i+1)] = 'https://www.sefaria.org/person/' + m['name'].replace(' ', '%20')
        if i > max_match:
            max_match = i
    rows += [item]
with open('people_talmud_matched.csv', 'wb') as fout:
    csv = unicodecsv.DictWriter(fout, ['Name'] + map(lambda x: 'Match Name {}'.format(x+1), xrange(max_match+1)) + map(lambda x: 'Match URL {}'.format(x+1), xrange(max_match+1)))
    csv.writeheader()
    csv.writerows(rows)
print max_match

14


In [18]:
# Sefaria unmatched
# TODO: 
rows = []
for yo in sef_mis_all:
    if not yo['matched']:
        rows += [
            {'Time Period': 'mishnah',
             'ID': yo['id'],
             'Hebrew Name': yo['heName'],
             'English Name': yo['name'],
             'URL': 'https://www.sefaria.org/person/' + yo['name'].replace(' ', '%20'),
             'jeLink': yo['jeLink'],
             'heWikiLink': yo['heWikiLink'],
             'enWikiLink': yo['enWikiLink'],
             'generation': yo['generation']
            }
        ]
for yo in sef_mis_all:
    if not yo['matched']:
        rows += [
            {'Time Period': 'talmud',
             'ID': yo['id'],
             'Hebrew Name': yo['heName'],
             'English Name': yo['name'],
             'URL': 'https://www.sefaria.org/person/' + yo['name'].replace(' ', '%20'),
             'jeLink': yo['jeLink'],
             'heWikiLink': yo['heWikiLink'],
             'enWikiLink': yo['enWikiLink'],
             'generation': yo['generation']
            }
        ]
with open('people_talmud_unmatched.csv', 'wb') as fout:
    csv = unicodecsv.DictWriter(fout, ['ID', 'Hebrew Name', 'English Name', 'URL', 'Time Period', 'jeLink', 'heWikiLink', 'enWikiLink', 'generation'])
    csv.writeheader()
    csv.writerows(rows)

In [120]:
# WikiData edges
rows = []
for sef_guy in sef_mis + sef_tal:
    for edge, val in sef_guy['edges'].items():
        for v in val:
            rows += [{'ID': sef_guy['_id'], 'Name': sef_guy.get('name', ''), 'He Name': sef_guy.get('heName', ''), 'Edge': edge, 'Value': v['name'], 'Value ID': v['id']}]
with open('people_talmud_edges.csv', 'wb') as fout:
    print 'yoyo'
    csv = unicodecsv.DictWriter(fout, ['ID', 'Name', 'He Name', 'Edge', 'Value', 'Value ID'])
    csv.writeheader()
    csv.writerows(rows)

yoyo


In [111]:
'Learned From'.lower()

'learned from'

In [89]:
print u'\u8206'

舆
